# Process US State-level Data from NYT

In [1]:
# Import packages
import pandas as pd
import numpy as np
from datetime import datetime
import geopandas as gpd

### Step0: Read data and get basic state groups

#### 0.0 Handle Report Data

Read updated CSV file

In [2]:
df_states_original = pd.read_csv("./us-states.csv", dtype={"date": np.string_,
                                                             "state": np.string_,
                                                             "fips": np.string_,
                                                            "cases": np.int32,
                                                            "deaths": np.int32})
df_states_original.sort_values("fips", ascending=True)

,date,state,fips,cases,deaths
5569,2020-06-12,Alabama,01,23710,769
3754,2020-05-10,Alabama,01,9889,393
1994,2020-04-08,Alabama,01,2498,67
6669,2020-07-02,Alabama,01,40111,985
10189,2020-09-04,Alabama,01,130393,2266
...,...,...,...,...,...
7213,2020-07-11,Virgin Islands,78,181,6
4738,2020-05-27,Virgin Islands,78,69,6
7268,2020-07-12,Virgin Islands,78,206,6
4298,2020-05-19,Virgin Islands,78,69,6


#### 0.1 Group Data to States

Transform cases/deaths data table to pivot table using fips as main index

In [3]:
pivot_states = pd.pivot_table(df_states_original , index=['state','fips'],
                       columns=['date'])
if pivot_states.isnull().values.any():
    pivot_states = pivot_states.replace(np.nan, 0).astype(int)
pivot_states.head(5)

cases                                              \
date            2020-01-21 2020-01-22 2020-01-23 2020-01-24 2020-01-25   
state      fips                                                          
Alabama    01            0          0          0          0          0   
Alaska     02            0          0          0          0          0   
Arizona    04            0          0          0          0          0   
Arkansas   05            0          0          0          0          0   
California 06            0          0          0          0          1   

                                                                        ...  \
date            2020-01-26 2020-01-27 2020-01-28 2020-01-29 2020-01-30  ...   
state      fips                                                         ...   
Alabama    01            0          0          0          0          0  ...   
Alaska     02            0          0          0          0          0  ...   
Arizona    04            1          1          1          1          1  ...   
Arkansas   05            0          0          0          0          0  ...   
California 06            2          2          2          2          2  ...   

                    deaths                                              \
date            2020-09-08 2020-09-09 2020-09-10 2020-09-11 2020-09-12   
state      fips                                                          
Alabama    01         2277       2285       2301       2333       2350   
Alaska     02           39         39         39         40         41   
Arizona    04         5221       5253       5275       5290       5316   
Arkansas   05          917        928        940        953        969   
California 06        13843      13990      14094      14265      14333   

                                                                        
date            2020-09-13 2020-09-14 2020-09-15 2020-09-16 2020-09-17  
state      fips                                                         
Alabama    01         2351       2355       2387       2392       2401  
Alaska     02           41         41         41         41         41  
Arizona    04         5322       5322       5346       5376       5411  
Arkansas   05          981        992       1150       1157       1166  
California 06        14386      14458      14606      14716      14807  

[5 rows x 482 columns]

#### 0.2 Get Date Info

Get dates from data table

In [4]:
date= pivot_states['cases'].columns
date

Index(['2020-01-21', '2020-01-22', '2020-01-23', '2020-01-24', '2020-01-25',
       '2020-01-26', '2020-01-27', '2020-01-28', '2020-01-29', '2020-01-30',
       ...
       '2020-09-08', '2020-09-09', '2020-09-10', '2020-09-11', '2020-09-12',
       '2020-09-13', '2020-09-14', '2020-09-15', '2020-09-16', '2020-09-17'],
      dtype='object', name='date', length=241)

Sort dates and get the start/end

In [5]:
date_str = np.sort(date)
dt_str_start=np.min(date_str)
dt_str_end=np.max(date_str)

Format dates and create a list

In [6]:
dt_start = datetime.strptime(dt_str_start, "%Y-%m-%d")
dt_end = datetime.strptime(dt_str_end, "%Y-%m-%d")
dt_range = pd.date_range(start=dt_start,end=dt_end)
print(len(dt_range), dt_range)
dt_range_str = list(map(lambda x: x.strftime("%Y-%m-%d"), dt_range.tolist()))
print(len(dt_range_str), dt_range_str)

241 DatetimeIndex(['2020-01-21', '2020-01-22', '2020-01-23', '2020-01-24',
               '2020-01-25', '2020-01-26', '2020-01-27', '2020-01-28',
               '2020-01-29', '2020-01-30',
               ...
               '2020-09-08', '2020-09-09', '2020-09-10', '2020-09-11',
               '2020-09-12', '2020-09-13', '2020-09-14', '2020-09-15',
               '2020-09-16', '2020-09-17'],
              dtype='datetime64[ns]', length=241, freq='D')
241 ['2020-01-21', '2020-01-22', '2020-01-23', '2020-01-24', '2020-01-25', '2020-01-26', '2020-01-27', '2020-01-28', '2020-01-29', '2020-01-30', '2020-01-31', '2020-02-01', '2020-02-02', '2020-02-03', '2020-02-04', '2020-02-05', '2020-02-06', '2020-02-07', '2020-02-08', '2020-02-09', '2020-02-10', '2020-02-11', '2020-02-12', '2020-02-13', '2020-02-14', '2020-02-15', '2020-02-16', '2020-02-17', '2020-02-18', '2020-02-19', '2020-02-20', '2020-02-21', '2020-02-22', '2020-02-23', '2020-02-24', '2020-02-25', '2020-02-26', '2020-02-27', '2020-02-

### Step1: Add Cases Time Series, First Case Date, Death Time Series, First Death Date

Create time series of cases/deaths

In [7]:
pivot_states['cases_ts'] = pivot_states['cases'].values.tolist()
pivot_states['cases_ts'] = pivot_states['cases_ts'].apply(lambda x: ','.join(map(str, x)))
pivot_states['deaths_ts'] = pivot_states['deaths'].values.tolist()
pivot_states['deaths_ts'] = pivot_states['deaths_ts'].apply(lambda x: ','.join(map(str, x)))

Get the date of first case/death

In [8]:
pivot_states['dt_first_case'] = (pivot_states['cases'] > 0).idxmax(axis=1)
pivot_states['dt_first_death'] = (pivot_states['deaths'] > 0).idxmax(axis=1)
# For death reports, deal with no deaths counties
pivot_states.loc[pivot_states['deaths'].iloc[:, -1] <= 0, 'dt_first_death'] = np.nan
pivot_states.head(5)

cases                                              \
date            2020-01-21 2020-01-22 2020-01-23 2020-01-24 2020-01-25   
state      fips                                                          
Alabama    01            0          0          0          0          0   
Alaska     02            0          0          0          0          0   
Arizona    04            0          0          0          0          0   
Arkansas   05            0          0          0          0          0   
California 06            0          0          0          0          1   

                                                                        ...  \
date            2020-01-26 2020-01-27 2020-01-28 2020-01-29 2020-01-30  ...   
state      fips                                                         ...   
Alabama    01            0          0          0          0          0  ...   
Alaska     02            0          0          0          0          0  ...   
Arizona    04            1          1          1          1          1  ...   
Arkansas   05            0          0          0          0          0  ...   
California 06            2          2          2          2          2  ...   

                    deaths                                              \
date            2020-09-12 2020-09-13 2020-09-14 2020-09-15 2020-09-16   
state      fips                                                          
Alabama    01         2350       2351       2355       2387       2392   
Alaska     02           41         41         41         41         41   
Arizona    04         5316       5322       5322       5346       5376   
Arkansas   05          969        981        992       1150       1157   
California 06        14333      14386      14458      14606      14716   

                                                                     cases_ts  \
date            2020-09-17                                                      
state      fips                                                                 
Alabama    01         2401  0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...   
Alaska     02           41  0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...   
Arizona    04         5411  0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...   
Arkansas   05         1166  0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...   
California 06        14807  0,0,0,0,1,2,2,2,2,2,3,3,6,6,6,6,6,6,6,6,7,7,7,...   

                                                         deaths_ts  \
date                                                                 
state      fips                                                      
Alabama    01    0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...   
Alaska     02    0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...   
Arizona    04    0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...   
Arkansas   05    0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...   
California 06    0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...   

                dt_first_case dt_first_death  
date                                          
state      fips                               
Alabama    01      2020-03-13     2020-03-25  
Alaska     02      2020-03-12     2020-03-27  
Arizona    04      2020-01-26     2020-03-20  
Arkansas   05      2020-03-11     2020-03-24  
California 06      2020-01-25     2020-03-04  

[5 rows x 486 columns]

### Step2: Add today_case, today_new_case, today_death, today_new_death

Get the latest case/death

In [9]:
yesterday = date.values[-2]
pivot_states['today_case'] = pivot_states['cases'][dt_str_end]
pivot_states['today_new_case'] = pivot_states['cases'][dt_str_end] - pivot_states['cases'][yesterday]
pivot_states['today_death'] = pivot_states['deaths'][dt_str_end]
pivot_states['today_new_death'] = pivot_states['deaths'][dt_str_end] - pivot_states['deaths'][yesterday]

### Step3: Read State Boundary GeoJSON file

Read GeoJson file

In [10]:
old_states_geojson_df = gpd.read_file(r"./states_update.geojson")
old_states_geojson_df.head(5)

,REGION,DIVISION,STATEFP,STATENS,GEOID,STUSPS,NAME,LSAD,MTFCC,FUNCSTAT,...,d2020-03-19,d2020-03-20,d2020-03-21,d2020-03-22,d2020-03-23,d2020-03-24,d2020-03-25,d2020-03-26,population,geometry
0,3,5,54,01779805,54,WV,West Virginia,00,G4000,A,...,0,0,0,0,0,0,0,0,1829054,"POLYGON ((-80.51899 40.63880, -80.51934 39.721..."
1,3,5,12,00294478,12,FL,Florida,00,G4000,A,...,1,1,2,2,4,3,0,6,20598139,"MULTIPOLYGON (((-82.98748 24.62538, -82.89993 ..."
2,2,3,17,01779784,17,IL,Illinois,00,G4000,A,...,3,1,1,1,5,4,3,7,12821497,"POLYGON ((-90.64284 42.50848, -87.01994 42.493..."
3,2,4,27,00662849,27,MN,Minnesota,00,G4000,A,...,0,0,1,0,0,0,0,1,5527358,"POLYGON ((-97.22872 49.00056, -95.15371 48.998..."
4,3,5,24,01714934,24,MD,Maryland,00,G4000,A,...,0,1,1,0,0,1,0,0,6003435,"POLYGON ((-77.71952 39.32131, -77.76086 39.340..."


### Step4: Extract necessary columns

Only keep the necessary columns

In [11]:
report_df = pivot_states[['cases_ts','deaths_ts','dt_first_case','dt_first_death','today_case','today_new_case', 'today_death','today_new_death']]
report_df = report_df.reset_index()
report_df.columns = ['state','fips','cases_ts','deaths_ts','dt_first_case','dt_first_death','today_case','today_new_case', 'today_death','today_new_death']
report_df.head(5)

,state,fips,cases_ts,deaths_ts,dt_first_case,dt_first_death,today_case,today_new_case,today_death,today_new_death
0,Alabama,01,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-03-13,2020-03-25,141757,670,2401,9
1,Alaska,02,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-03-12,2020-03-27,7413,105,41,0
2,Arizona,04,"0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-01-26,2020-03-20,211663,1752,5411,35
3,Arkansas,05,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-03-11,2020-03-24,73211,992,1166,9
4,California,06,"0,0,0,0,1,2,2,2,2,2,3,3,6,6,6,6,6,6,6,6,7,7,7,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-01-25,2020-03-04,776944,3626,14807,91


### Step5: Assign geometries to states

Stay consistency with geoJson states

In [12]:
report_df = report_df.replace('Northern Mariana Islands','Commonwealth of the Northern Mariana Islands')

Merge case dataframe and geometry

In [13]:
final_df = pd.merge(old_states_geojson_df[["NAME", "population", "geometry"]], report_df, how='left', left_on=['NAME'], right_on = ['state'])
final_df.columns

Index(['NAME', 'population', 'geometry', 'state', 'fips', 'cases_ts',
       'deaths_ts', 'dt_first_case', 'dt_first_death', 'today_case',
       'today_new_case', 'today_death', 'today_new_death'],
      dtype='object')

### Step6: Finalize the dataframe

Keep and add essential columns

In [14]:
final_df=final_df[["NAME", "population", "fips", "dt_first_case", "dt_first_death", "cases_ts", "deaths_ts", 'today_case','today_new_case', 'today_death','today_new_death', "geometry"]]
final_df['start'] = dt_str_start
final_df['end'] = dt_str_end
final_df['dt_unit'] = "day"
final_df.head(5)

,NAME,population,fips,dt_first_case,dt_first_death,cases_ts,deaths_ts,today_case,today_new_case,today_death,today_new_death,geometry,start,end,dt_unit
0,West Virginia,1829054,54,2020-03-17,2020-03-29,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",13430.0,234.0,294.0,4.0,"POLYGON ((-80.51899 40.63880, -80.51934 39.721...",2020-01-21,2020-09-17,day
1,Florida,20598139,12,2020-03-01,2020-03-06,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",674448.0,3255.0,13085.0,147.0,"MULTIPOLYGON (((-82.98748 24.62538, -82.89993 ...",2020-01-21,2020-09-17,day
2,Illinois,12821497,17,2020-01-24,2020-03-17,"0,0,0,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",271370.0,2088.0,8643.0,21.0,"POLYGON ((-90.64284 42.50848, -87.01994 42.493...",2020-01-21,2020-09-17,day
3,Minnesota,5527358,27,2020-03-06,2020-03-21,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",86774.0,909.0,1994.0,9.0,"POLYGON ((-97.22872 49.00056, -95.15371 48.998...",2020-01-21,2020-09-17,day
4,Maryland,6003435,24,2020-03-05,2020-03-18,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",119017.0,630.0,3861.0,6.0,"POLYGON ((-77.71952 39.32131, -77.76086 39.340...",2020-01-21,2020-09-17,day


Check Samoa data

In [15]:
final_df[final_df['NAME'] == 'American Samoa']

,NAME,population,fips,dt_first_case,dt_first_death,cases_ts,deaths_ts,today_case,today_new_case,today_death,today_new_death,geometry,start,end,dt_unit
41,American Samoa,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((-171.14118 -11.04661, -171.111...",2020-01-21,2020-09-17,day


New York Times removed Samoa data since 4/27

In [16]:
final_df = final_df.drop(final_df[final_df['NAME'] == 'American Samoa'].index)
final_df = final_df.reset_index()
final_df

,index,NAME,population,fips,dt_first_case,dt_first_death,cases_ts,deaths_ts,today_case,today_new_case,today_death,today_new_death,geometry,start,end,dt_unit
0,0,West Virginia,1829054,54,2020-03-17,2020-03-29,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",13430.0,234.0,294.0,4.0,"POLYGON ((-80.51899 40.63880, -80.51934 39.721...",2020-01-21,2020-09-17,day
1,1,Florida,20598139,12,2020-03-01,2020-03-06,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",674448.0,3255.0,13085.0,147.0,"MULTIPOLYGON (((-82.98748 24.62538, -82.89993 ...",2020-01-21,2020-09-17,day
2,2,Illinois,12821497,17,2020-01-24,2020-03-17,"0,0,0,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",271370.0,2088.0,8643.0,21.0,"POLYGON ((-90.64284 42.50848, -87.01994 42.493...",2020-01-21,2020-09-17,day
3,3,Minnesota,5527358,27,2020-03-06,2020-03-21,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",86774.0,909.0,1994.0,9.0,"POLYGON ((-97.22872 49.00056, -95.15371 48.998...",2020-01-21,2020-09-17,day
4,4,Maryland,6003435,24,2020-03-05,2020-03-18,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",119017.0,630.0,3861.0,6.0,"POLYGON ((-77.71952 39.32131, -77.76086 39.340...",2020-01-21,2020-09-17,day
5,5,Rhode Island,1056611,44,2020-03-01,2020-03-28,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",23488.0,130.0,1085.0,4.0,"MULTIPOLYGON (((-71.67881 41.15891, -71.63497 ...",2020-01-21,2020-09-17,day
6,6,Idaho,1687809,16,2020-03-13,2020-03-26,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",36924.0,330.0,435.0,5.0,"POLYGON ((-116.91607 45.99536, -116.98259 46.0...",2020-01-21,2020-09-17,day
7,7,New Hampshire,1343622,33,2020-03-02,2020-03-23,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",7814.0,34.0,438.0,0.0,"POLYGON ((-71.50108 45.01335, -71.49804 45.070...",2020-01-21,2020-09-17,day
8,8,North Carolina,10155624,37,2020-03-03,2020-03-25,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",190307.0,1507.0,3208.0,32.0,"POLYGON ((-81.67749 36.58814, -80.12217 36.542...",2020-01-21,2020-09-17,day
9,9,Vermont,624977,50,2020-03-07,2020-03-19,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",1705.0,3.0,58.0,0.0,"POLYGON ((-73.34312 45.01084, -71.46505 45.013...",2020-01-21,2020-09-17,day


Calculate weekly change rates of cases and form a time-series list

In [17]:
final_df['change_ts'] = ""
for x in range(0, len(final_df)):
    newCaseList = [0]
    changeRateList = []
    caseStrList = final_df.loc[x,'cases_ts'].split(",")
    for i in range(1, len(caseStrList)):
        dailyNewCase = float(caseStrList[i]) - float(caseStrList[i-1])
        newCaseList.append(dailyNewCase)
    for i in range(0, len(newCaseList)):
        if i < 13:
            changeRate = 0
        else:
            currentWeekSum = 0
            previousWeekSum = 0
            for j in range(0,7):
                currentWeekSum = currentWeekSum + newCaseList[i-j]
            for k in range(7,14):
                previousWeekSum = previousWeekSum + newCaseList[i-k]
            if previousWeekSum == 0 and currentWeekSum == 0:
                changeRate = 0
            elif previousWeekSum == 0 and currentWeekSum != 0:
                changeRate = currentWeekSum
            else:
                changeRate = round(currentWeekSum/previousWeekSum-1,2)
        changeRateList.append(changeRate)
        changeRateStrList = list(map(str, changeRateList))
#     print(','.join(changeRateStrList))
    final_df.loc[x,'change_ts'] = ','.join(changeRateStrList)

final_df

,index,NAME,population,fips,dt_first_case,dt_first_death,cases_ts,deaths_ts,today_case,today_new_case,today_death,today_new_death,geometry,start,end,dt_unit,change_ts
0,0,West Virginia,1829054,54,2020-03-17,2020-03-29,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",13430.0,234.0,294.0,4.0,"POLYGON ((-80.51899 40.63880, -80.51934 39.721...",2020-01-21,2020-09-17,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."
1,1,Florida,20598139,12,2020-03-01,2020-03-06,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",674448.0,3255.0,13085.0,147.0,"MULTIPOLYGON (((-82.98748 24.62538, -82.89993 ...",2020-01-21,2020-09-17,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."
2,2,Illinois,12821497,17,2020-01-24,2020-03-17,"0,0,0,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",271370.0,2088.0,8643.0,21.0,"POLYGON ((-90.64284 42.50848, -87.01994 42.493...",2020-01-21,2020-09-17,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,-1.0,-1...."
3,3,Minnesota,5527358,27,2020-03-06,2020-03-21,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",86774.0,909.0,1994.0,9.0,"POLYGON ((-97.22872 49.00056, -95.15371 48.998...",2020-01-21,2020-09-17,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."
4,4,Maryland,6003435,24,2020-03-05,2020-03-18,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",119017.0,630.0,3861.0,6.0,"POLYGON ((-77.71952 39.32131, -77.76086 39.340...",2020-01-21,2020-09-17,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."
5,5,Rhode Island,1056611,44,2020-03-01,2020-03-28,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",23488.0,130.0,1085.0,4.0,"MULTIPOLYGON (((-71.67881 41.15891, -71.63497 ...",2020-01-21,2020-09-17,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."
6,6,Idaho,1687809,16,2020-03-13,2020-03-26,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",36924.0,330.0,435.0,5.0,"POLYGON ((-116.91607 45.99536, -116.98259 46.0...",2020-01-21,2020-09-17,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."
7,7,New Hampshire,1343622,33,2020-03-02,2020-03-23,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",7814.0,34.0,438.0,0.0,"POLYGON ((-71.50108 45.01335, -71.49804 45.070...",2020-01-21,2020-09-17,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."
8,8,North Carolina,10155624,37,2020-03-03,2020-03-25,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",190307.0,1507.0,3208.0,32.0,"POLYGON ((-81.67749 36.58814, -80.12217 36.542...",2020-01-21,2020-09-17,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."
9,9,Vermont,624977,50,2020-03-07,2020-03-19,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",1705.0,3.0,58.0,0.0,"POLYGON ((-73.34312 45.01084, -71.46505 45.013...",2020-01-21,2020-09-17,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."


### Step 7: Output file

Save file

In [18]:
final_df.to_file(r"./nyt_states_data.geojson", driver='GeoJSON', encoding='utf-8')
print("done")

done
